<a href="https://colab.research.google.com/github/m-hefnawy/Pytorch-Tutorial-YouTube/blob/main/Intro_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib as plt
%matplotlib inline

In [3]:
# Create a Model that inherits nn.Module to classify iris flowers (four features, three classes)
class Model(nn.Module):
  # Input layer (4 features of the flower) -->
  # Hidden layer1 (number of neurons) -->
  # Hidden layer2 (number of neurons) -->
  # Output layer (3 classes)
  def __init__(self, in_features=4, h1=8, h2= 16, out_features=3):
    super().__init__() # inistintiate the nn.Module
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

In [5]:
# Pick a manual seed for randomization
torch.manual_seed(41)
# Create an instance of the model
model = Model()
# Load iris data

iris = datasets.load_iris()

my_df = pd.DataFrame(
    iris.data,
    columns=iris.feature_names
    )

my_df['variety'] = iris.target
my_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
